In [1]:
from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
import pytorch_lightning as pl
from tqdm.notebook import tqdm

In [2]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
gloveFile = 'data/conll03/trimmed.300d.Cased.txt'
symbFile = 'data/conll03/sym.glove'
data = NERDataset(sourceName, targetName, gloveFile, symbFile)
loader = data.getLoader(4096)

In [3]:
max(data.wordIdx.values()), len(data.wordIdx)

(21387, 21388)

In [4]:
numChars = 100
charEmbedding = 128
numWords = len(data.wordIdx)
wordEmbedding = 300
contextOutputUnits = 128
contextTransitionNumber = transitionNumber = 4
encoderUnits = 256
decoderUnits = 256

In [5]:
model = GlobalContextualDeepTransition(numChars, charEmbedding, numWords,
                     wordEmbedding, contextOutputUnits, contextTransitionNumber,
                        encoderUnits, decoderUnits, transitionNumber, data.numTags)

# add glove weights here
model.init_weights(data.embeddingWeights)

numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {numParams:,}") # 6,788,797 but now 7,313,341 !

Trainable parameters: 7,313,341


In [6]:
trainer = pl.Trainer(gradient_clip_val=5., gpus=1, max_epochs=1)
trainer.fit(model, loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                           | Params
--------------------------------------------------------------------
0 | contextEncoder   | GlobalContextualEncoder        | 7 M   
1 | sequenceLabeller | SequenceLabelingEncoderDecoder | 6 M   
Epoch 0: 100%|██████████| 50/50 [04:27<00:00,  5.35s/it, loss=1.010, v_num=15]


1